In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import xgboost
import numpy as np
import seaborn as sns
from sklearn.ensemble import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [184]:
dataset = sns.load_dataset("titanic")
dataset=dataset.drop('embark_town',axis= 1)
dataset = dataset.drop('alive', axis=1)
dataset= dataset.drop('who', axis=1)
dataset= dataset.drop('adult_male', axis=1)
dataset= dataset.drop('pclass', axis=1)
dataset= dataset.drop('deck', axis=1)
dataset= dataset.drop('parch', axis=1)
dataset= dataset.drop('sibsp', axis=1)

dataset['age']=dataset['age'].fillna(dataset['age'].mean())
dataset['sex'] = dataset['sex'].map( {'female': 1, 'male': 0} ).astype(int)
dataset['class'] = dataset['class'].map( {'Third': 3, 'Second': 2, 'First': 1} ).astype(int)
dataset['alone'] = dataset['alone'].map( {True: 1, False: 0} ).astype(int)

dataset['age_band'] = pd.cut(dataset['age'], 5)
dataset.loc[ dataset['age'] <= 16, 'age'] = 0
dataset.loc[(dataset['age'] > 16) & (dataset['age'] <= 32), 'age'] = 1
dataset.loc[(dataset['age'] > 32) & (dataset['age'] <= 48), 'age'] = 2
dataset.loc[(dataset['age'] > 48) & (dataset['age'] <= 64), 'age'] = 3
dataset.loc[ dataset['age'] > 64, 'age']
dataset = dataset.drop(['age_band'], axis=1)

freq_port  = dataset.embarked.dropna().mode()[0]
dataset['embarked'] = dataset['embarked'].fillna(freq_port)
dataset['embarked'] = dataset['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)


dataset['fare'].fillna(dataset['fare'].dropna().median(), inplace=True)
dataset['FareBand'] = pd.qcut(dataset['fare'], 4)
dataset.loc[ dataset['fare'] <= 7.91, 'fare'] = 0
dataset.loc[(dataset['fare'] > 7.91) & (dataset['fare'] <= 14.454), 'fare'] = 1
dataset.loc[(dataset['fare'] > 14.454) & (dataset['fare'] <= 31), 'fare']   = 2
dataset.loc[ dataset['fare'] > 31, 'fare'] = 3
dataset['fare'] = dataset['fare'].astype(int)
dataset = dataset.drop(['FareBand'], axis=1)

dataset['age*class'] = dataset.age * dataset['class']


dataset.head()

X = dataset.drop("survived", axis=1)
y = dataset['survived']

print(dataset.head(), '\n')

for size in [0.01, 0.15, 0.5]:
    print(size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=42)
    model_xgb = xgboost.XGBClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)

    models = (
        (xgboost.XGBClassifier, {'n_estimators':100}),
        (RandomForestClassifier, {'n_estimators':100}),
        (MLPClassifier, {'solver': 'lbfgs', 'alpha': 1e-5, 'max_iter': 50000})
    )

    for model_class, kwargs in models:
        model = model_class(random_state=42, **kwargs).fit(X_train, y_train)
        print(model.__class__.__name__)
        model_train = round(model.score(X_train, y_train), 5)
        model_test = round(model.score(X_test, y_test), 5)
        print(model_train, model_test)
    
    
    print(50*"_")

   survived  sex  age  fare  embarked  class  alone  age*class
0         0    0  1.0     0         0      3      0        3.0
1         1    1  2.0     3         1      1      0        2.0
2         1    1  1.0     1         0      3      1        3.0
3         1    1  2.0     3         0      1      0        2.0
4         0    0  2.0     1         0      3      1        6.0 

0.01
XGBClassifier
0.85601 0.77778
RandomForestClassifier
0.85714 0.77778
MLPClassifier
0.85714 0.88889
__________________________________________________
0.15
XGBClassifier
0.85997 0.79851
RandomForestClassifier
0.86129 0.80597
MLPClassifier
0.86129 0.80597
__________________________________________________
0.5
XGBClassifier
0.87865 0.78475
RandomForestClassifier
0.87865 0.78924
MLPClassifier
0.87865 0.78924
__________________________________________________


In [145]:
xgboost.XGBClassifier().__class__

xgboost.sklearn.XGBClassifier

In [189]:
dataset = sns.load_dataset("titanic")
dataset=dataset.drop('embark_town',axis= 1)
dataset = dataset.drop('alive', axis=1)
dataset= dataset.drop('who', axis=1)
dataset= dataset.drop('deck', axis=1)
dataset= dataset.drop('parch', axis=1)

dataset['age']=dataset['age'].fillna(dataset['age'].mean())
dataset['sex'] = dataset['sex'].map( {'female': 1, 'male': 0} ).astype(int)
dataset['class'] = dataset['class'].map( {'Third': 3, 'Second': 2, 'First': 1} ).astype(int)
dataset['alone'] = dataset['alone'].map( {True: 1, False: 0} ).astype(int)

# dataset= dataset.drop('adult_male', axis=1)
dataset['adult_male'] = dataset['adult_male'].map( {True: 1, False: 0} ).astype(int)

freq_port  = dataset.embarked.dropna().mode()[0]
dataset['embarked'] = dataset['embarked'].fillna(freq_port)
dataset['embarked'] = dataset['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

dataset.head()

X = dataset.drop("survived", axis=1)
y = dataset['survived']

print(dataset.head(), '\n')

for size in [0.01, 0.15, 0.2,]:
    print(size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=42)
    model_xgb = xgboost.XGBClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)

    models = (
        (xgboost.XGBClassifier, {'n_estimators':1}),
        (RandomForestClassifier, {'n_estimators':1}),
        (RandomForestClassifier, {'n_estimators':1, 'bootstrap': False}),
        # (MLPClassifier, {'solver': 'lbfgs', 'alpha': 1e-5, 'max_iter': 50000})
    )

    for model_class, kwargs in models:
        model = model_class(random_state=42, **kwargs).fit(X_train, y_train)
        print(model.__class__.__name__)
        model_train = round(model.score(X_train, y_train), 5)
        model_test = round(model.score(X_test, y_test), 5)
        print(model_train, model_test)
    
    
    print(50*"_")

   survived  pclass  sex   age  sibsp     fare  embarked  class  adult_male  \
0         0       3    0  22.0      1   7.2500         0      3           1   
1         1       1    1  38.0      1  71.2833         1      1           0   
2         1       3    1  26.0      0   7.9250         0      3           0   
3         1       1    1  35.0      1  53.1000         0      1           0   
4         0       3    0  35.0      0   8.0500         0      3           1   

   alone  
0      0  
1      0  
2      1  
3      0  
4      1   

0.01
XGBClassifier
0.85601 0.77778
RandomForestClassifier
0.89796 0.77778
RandomForestClassifier
0.98186 1.0
__________________________________________________
0.15
XGBClassifier
0.86658 0.83582
RandomForestClassifier
0.91017 0.79851
RandomForestClassifier
0.98018 0.81343
__________________________________________________
0.2
XGBClassifier
0.87079 0.83799
RandomForestClassifier
0.90449 0.75419
RandomForestClassifier
0.98034 0.81006
_____________________

In [190]:
from shap import GPUTreeExplainer

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [195]:
e = GPUTreeExplainer(model_xgb)
e.shap_values(X_train)

cuda extension was not built during install!


ImportError: cannot import name '_cext_gpu' from partially initialized module 'shap' (most likely due to a circular import) (D:\AMI\venv\Lib\site-packages\shap\__init__.py)